In [33]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


dataset_path_folder = './data/physical_exercise_dataset'
preprocessed_data_file = './data/preprocessed_dataset/train.csv'

import os
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./data/test_exercises/test.mp4
./data/physical_exercise_dataset/3d_distances.csv
./data/physical_exercise_dataset/landmarks.csv
./data/physical_exercise_dataset/labels.csv
./data/physical_exercise_dataset/angles.csv
./data/physical_exercise_dataset/xyz_distances.csv
./data/preprocessed_dataset/train.csv


In [34]:
df = pd.read_csv(preprocessed_data_file)
df.head()

,pose_id,pose,x_nose,y_nose,z_nose,x_left_eye_inner,y_left_eye_inner,z_left_eye_inner,x_left_eye,y_left_eye,...,z_left_heel,x_right_heel,y_right_heel,z_right_heel,x_left_foot_index,y_left_foot_index,z_left_foot_index,x_right_foot_index,y_right_foot_index,z_right_foot_index
0,0,squats_up,-0.382815,-48.231250,-54.405792,0.137189,-50.040543,-51.997875,0.502047,-50.058890,...,33.283375,-12.302963,52.377000,21.567219,16.609533,54.351162,15.890617,-16.779694,55.382572,2.050140
1,1,situp_down,54.146880,-12.822491,5.564175,56.762527,-11.221117,-0.363063,56.795986,-10.608183,...,-8.073100,-37.519974,4.045297,62.104233,-60.468560,6.281939,-8.185631,-52.589350,-3.852214,59.750070
2,2,situp_down,9.891440,-54.147266,85.344970,12.784414,-55.229970,88.534775,14.006874,-54.291880,...,-132.024460,-61.214890,53.040688,-16.786700,-5.844672,99.829030,-166.543060,-73.826830,38.470097,-40.519830
3,3,jumping_jacks_up,0.904673,-51.350130,-33.606970,1.338871,-53.172337,-30.013737,1.743913,-53.050697,...,51.615970,-15.543400,38.251835,34.033176,16.610210,46.572014,38.680614,-18.784650,45.098710,20.343110
4,4,jumping_jacks_down,-3.153129,-55.255062,-17.745928,-2.046205,-57.477790,-18.198952,-1.506304,-57.428230,...,-2.849554,2.811829,56.000500,56.605590,0.566345,66.231050,-19.252045,-3.451297,60.029484,45.503677


In [35]:
df['pose'].value_counts()

pose
jumping_jacks_down    151
jumping_jacks_up      145
pullups_down          123
pushups_up            115
squats_up             111
pullups_up            108
squats_down           101
situp_down             82
pushups_down           82
situp_up               79
Name: count, dtype: int64

In [36]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split

In [37]:
encoder = LabelEncoder()
y = df['pose']
y = encoder.fit_transform(y)

In [38]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
class_weights

array([0.72649007, 0.75655172, 0.89186992, 1.01574074, 1.33780488,
       0.95391304, 1.33780488, 1.38860759, 1.08613861, 0.98828829])

In [39]:
scaler = MinMaxScaler()

In [40]:
X = df.drop(['pose_id', 'pose'], axis='columns')
X

,x_nose,y_nose,z_nose,x_left_eye_inner,y_left_eye_inner,z_left_eye_inner,x_left_eye,y_left_eye,z_left_eye,x_left_eye_outer,...,z_left_heel,x_right_heel,y_right_heel,z_right_heel,x_left_foot_index,y_left_foot_index,z_left_foot_index,x_right_foot_index,y_right_foot_index,z_right_foot_index
0,-0.382815,-48.231250,-54.405792,0.137189,-50.040543,-51.997875,0.502047,-50.058890,-51.986694,0.858925,...,33.283375,-12.302963,52.377000,21.567219,16.609533,54.351162,15.890617,-16.779694,55.382572,2.050140
1,54.146880,-12.822491,5.564175,56.762527,-11.221117,-0.363063,56.795986,-10.608183,-0.378148,56.832386,...,-8.073100,-37.519974,4.045297,62.104233,-60.468560,6.281939,-8.185631,-52.589350,-3.852214,59.750070
2,9.891440,-54.147266,85.344970,12.784414,-55.229970,88.534775,14.006874,-54.291880,88.543910,15.034156,...,-132.024460,-61.214890,53.040688,-16.786700,-5.844672,99.829030,-166.543060,-73.826830,38.470097,-40.519830
3,0.904673,-51.350130,-33.606970,1.338871,-53.172337,-30.013737,1.743913,-53.050697,-30.007776,2.137811,...,51.615970,-15.543400,38.251835,34.033176,16.610210,46.572014,38.680614,-18.784650,45.098710,20.343110
4,-3.153129,-55.255062,-17.745928,-2.046205,-57.477790,-18.198952,-1.506304,-57.428230,-18.204160,-0.933045,...,-2.849554,2.811829,56.000500,56.605590,0.566345,66.231050,-19.252045,-3.451297,60.029484,45.503677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,-25.679585,-47.380875,-5.901453,-25.139788,-51.002510,-10.440426,-24.879524,-51.218052,-10.440801,-24.600859,...,14.827999,-55.717426,-0.799386,48.414420,-59.586680,-16.158272,1.463939,-56.319120,-17.240007,40.646820
1093,-1.185803,-51.386070,-31.526268,-0.436185,-53.642360,-28.797546,0.175695,-53.750496,-28.796741,0.790328,...,8.130944,-12.678401,47.509556,62.822067,17.710821,51.948390,-14.776978,-17.019627,49.782352,47.884693
1094,-4.307419,-49.337822,7.097422,-4.982467,-51.214745,9.683287,-5.098945,-51.260090,9.667620,-5.205800,...,-22.450487,35.971850,14.499550,-48.723770,36.879860,-5.513745,-34.270077,39.437458,0.077004,-62.674248
1095,-41.915108,-1.429882,-64.905620,-43.944553,-5.871200,-69.808044,-43.605865,-7.228406,-69.804830,-43.263298,...,-111.222755,-39.931140,4.812771,-106.005490,15.440416,-9.404089,-123.269000,-52.717530,-15.088408,-111.827450


In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True, random_state=2022)

In [42]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [43]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler

In [44]:
class ExerciseDataset(Dataset):
    def __init__(self, X, y):
        self.x = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.int64))
        self.n_samples = X.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.n_samples

In [45]:
train_dataset = ExerciseDataset(X_train, y_train)

In [46]:
batch_size = 50

In [47]:
train_loader = DataLoader(train_dataset, batch_size=batch_size)

In [48]:
from torch import nn

In [49]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

In [50]:
epochs = 40
learning_rate = 0.01
hidden_size = 200

In [51]:
model = NeuralNet(X_train.shape[1], hidden_size, len(class_weights))

In [52]:
criterion = nn.CrossEntropyLoss(weight=torch.from_numpy(class_weights.astype(np.float32)))
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [53]:
n_total_steps = len(train_loader)

In [54]:
for epoch in range(epochs):
    for i, (features, labels) in enumerate(train_loader):
        outputs = model(features)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 6 == 0:
            print(f'epoch {epoch+1} / {epochs}, step {i+1} / {n_total_steps}, loss = {loss.item():.4f}')

epoch 1 / 40, step 6 / 18, loss = 1.9939
epoch 1 / 40, step 12 / 18, loss = 1.8506
epoch 1 / 40, step 18 / 18, loss = 1.9194
epoch 2 / 40, step 6 / 18, loss = 1.4209
epoch 2 / 40, step 12 / 18, loss = 1.4399
epoch 2 / 40, step 18 / 18, loss = 1.5199
epoch 3 / 40, step 6 / 18, loss = 1.1054
epoch 3 / 40, step 12 / 18, loss = 1.2718
epoch 3 / 40, step 18 / 18, loss = 1.3858
epoch 4 / 40, step 6 / 18, loss = 0.9643
epoch 4 / 40, step 12 / 18, loss = 1.1724
epoch 4 / 40, step 18 / 18, loss = 1.2936
epoch 5 / 40, step 6 / 18, loss = 0.8872
epoch 5 / 40, step 12 / 18, loss = 1.0907
epoch 5 / 40, step 18 / 18, loss = 1.2351
epoch 6 / 40, step 6 / 18, loss = 0.8559
epoch 6 / 40, step 12 / 18, loss = 1.0180
epoch 6 / 40, step 18 / 18, loss = 1.2120
epoch 7 / 40, step 6 / 18, loss = 0.8200
epoch 7 / 40, step 12 / 18, loss = 0.9619
epoch 7 / 40, step 18 / 18, loss = 1.2010
epoch 8 / 40, step 6 / 18, loss = 0.7633
epoch 8 / 40, step 12 / 18, loss = 0.9324
epoch 8 / 40, step 18 / 18, loss = 1.1423


In [55]:
test_features = torch.from_numpy(X_test.astype(np.float32))
test_labels = y_test
with torch.no_grad():
    outputs = model(test_features)
    _, predictions = torch.max(outputs, 1)

In [56]:
from sklearn.metrics import classification_report

In [57]:
print(classification_report(test_labels, predictions, target_names=encoder.classes_))

                    precision    recall  f1-score   support

jumping_jacks_down       0.88      0.77      0.82        30
  jumping_jacks_up       0.81      0.72      0.76        29
      pullups_down       0.68      0.84      0.75        25
        pullups_up       0.75      0.55      0.63        22
      pushups_down       0.93      0.88      0.90        16
        pushups_up       0.92      0.96      0.94        23
        situp_down       0.89      0.94      0.91        17
          situp_up       0.52      0.75      0.62        16
       squats_down       1.00      0.60      0.75        20
         squats_up       0.52      0.68      0.59        22

          accuracy                           0.76       220
         macro avg       0.79      0.77      0.77       220
      weighted avg       0.79      0.76      0.77       220



In [58]:
# # stop it
# import cv2
# import mediapipe as mp

# mp_drawing = mp.solutions.drawing_utils
# mp_pose = mp.solutions.pose

# # Initialize MediaPipe Pose model
# pose_model = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# # Load video
# video_path = 'revive/test.mp4'  # Replace with your video file path
# video = cv2.VideoCapture(video_path)

# # Get video resolution
# width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# # Initialize figure for plotting keypoints
# import matplotlib.pyplot as plt
# fig, ax = plt.subplots(figsize=(16, 9))

# # Define list to store keypoints
# keypoints_list = []

# # Process video frames
# while True:
#     # Read frame
#     ret, frame = video.read()

#     if not ret:
#         break

#     # Flip the image horizontally for a more natural selfie-view.
#     frame = cv2.flip(frame, 1)

#     # Convert the image to RGB format for MediaPipe
#     image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#     # Make a detection
#     results = pose_model.process(image)

#     # Extract keypoints
#     keypoints = []
#     if results.pose_landmarks is not None:
#         for landmark in results.pose_landmarks.landmark:
#             keypoints.append([landmark.x, landmark.y, landmark.z])

#     # Store keypoints in list
#     keypoints_list.append(keypoints)

#     # Draw pose landmarks on image
#     if results.pose_landmarks is not None:
#         mp_drawing.draw_landmarks(
#             image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

#     # Display the resulting image
#     plt.imshow(image)
#     plt.show(block=False)
#     plt.pause(0.01)

# # Close the video
# video.release()

# # Plot keypoints
# keypoints_array = np.array(keypoints_list)
# fig, ax = plt.subplots(figsize=(16, 9))
# ax.plot(keypoints_array[:, :, 0], keypoints_array[:, :, 1])
# ax.set_xlabel('Frame')
# ax.set_ylabel('Pixel Coordinate')
# plt.show()


In [59]:
torch.save(model.state_dict(), './models/model.pt')

In [60]:
# Define and train a single model for all exercises
model = NeuralNet(X_train.shape[1], hidden_size, len(class_weights))
criterion = nn.CrossEntropyLoss(weight=torch.from_numpy(class_weights.astype(np.float32)))
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    for i, (features, labels) in enumerate(train_loader):
        outputs = model(features)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Save the model
torch.save(model.state_dict(), './models/all_exercises_model.pt')